<a href="https://colab.research.google.com/github/michaelbejusca/TextGenerator_MarkovModel/blob/main/4032GENAIY_Text_Generation_with_a_Markov_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Generation with a Markov Model

Using Markov models/chains is one of the oldest methods for generating text.

This notebook demonstrates a very simple way to produce a Markov Chain from some source text, based on constructing character-level n-grams, and then use this to generate some new text, hopefully in the same style.

In [ ]:
input_text = "Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?' So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her. There was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, `Oh dear! Oh dear! I shall be late!' (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before seen a rabbit with either a waistcoat-pocket, or a watch to take out of it, and burning with curiosity, she ran across the field after it, and fortunately was just in time to see it pop down a large rabbit-hole under the hedge."

We can display an extract of the input text and format it for the screen using `textwrap`:

In [ ]:
import textwrap
print(textwrap.fill(input_text, 120))

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice
she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use
of a book,' thought Alice `without pictures or conversation?' So she was considering in her own mind (as well as she
could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be
worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.
There was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say
to itself, `Oh dear! Oh dear! I shall be late!' (when she thought it over afterwards, it occurred to her that she ought
to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of
its waistcoat-pocket, and looked at it, and the

## What is a Markov Chain?
A Markov Chain is a stochastic process that is "memory-less". That is, the probability of *future* states are not dependent upon the states that preceded the *present* state.

The general property of a Markov Chain is:

\begin{equation}
P(X_{n+1} = j \mid X_0 = i_0, X_1 = i_1, \dots X_{n-1} = i_{n-1}, X_n = i) = P(X_{n+1} = j \mid X_n = i)
\end{equation}

## Step 1: Create a Dictionary to Capture Transition Probabilities

To implement a Markov chain, we are going to create a **dictionary of n-grams**. The n-grams in this model will be based on characters, not words, in the input text.

For the moment, we will construct a Markov chain using n-grams or length 3, or trigrams. But the order of the n-grams is a parameter that can be tuned later get better (or more interesting) generated texts.

The first step is to divide the text into n-grams. For this dictionary, the n-grams will be the keys and each instance of a character that follows a given n-gram will be added to an array as the value for the key.

This means that if a combination of n-gram and a specific character appear twice in the input text, then the n-gram's array will contain the same character twice. This captures the frequency with which a character follows a n-gram, but perhaps not in the most elegant, or space-efficient, way.

In [ ]:
def markov_process(seq, order): #function creating a dictionary
  markov_dict = {} #empty dictionary to store the model
  seq = list(seq[:]) + [None] # convert the text into list of characters
  # order is the number of grams we are using.
  for i in range(len(seq) - order):
      gram = tuple(seq[i:i+order])
      if gram not in markov_dict:
          markov_dict[gram] = []
      markov_dict[gram].append(seq[i+order])
  return markov_dict

Naturally, we can inspect the output of the dictionary:

In [ ]:
markov_dict = markov_process(input_text, 3)
print(textwrap.fill(str(markov_dict)[:960], 120))

{('A', 'l', 'i'): ['c', 'c', 'c', 'c'], ('l', 'i', 'c'): ['e', 'e', 'e', 'e'], ('i', 'c', 'e'): [' ', ' ', ' ', ' ', '
'], ('c', 'e', ' '): ['w', 'o', 's', '`', 't', 's'], ('e', ' ', 'w'): ['a', 'a', 'o', 'a', 'a', 'o'], (' ', 'w', 'a'):
['s', 's', 's', 's', 'y', 't', 'i', 'i', 't', 's'], ('w', 'a', 's'): [' ', ' ', ' ', ' ', ' '], ('a', 's', ' '): ['b',
'r', 'c', 'w', 's', 'n', 'j'], ('s', ' ', 'b'): ['e'], (' ', 'b', 'e'): ['g', ' ', ' ', 'f'], ('b', 'e', 'g'): ['i'],
('e', 'g', 'i'): ['n'], ('g', 'i', 'n'): ['n'], ('i', 'n', 'n'): ['i'], ('n', 'n', 'i'): ['n'], ('n', 'i', 'n'): ['g',
'g'], ('i', 'n', 'g'): [' ', ' ', ' ', ' ', ',', ' ', ' ', ' ', ' ', ' ', ' '], ('n', 'g', ' '): ['t', 'b', 'n', 't',
'i', 'a', 'u', 't', 's', 'w'], ('g', ' ', 't'): ['o', 'o', 'h'], (' ', 't', 'o'): [' ', ' ', ' ', ' ', ' ', ' ', 'o', '
', ' ', ' '], ('t', 'o', ' '): ['g', 'd', 't', 'h', 'i', 'h', 'h', 'h', 't', 's'], ('o', ' ', 'g'): ['e'], (' ', 'g',
'e'): ['t'


## Step 2: Generate Text

After defining the states of the Markov chain the n-gram model can be used generate text. By default the starting point for generation is the first n-gram in the text. This is asigned to `currentgram` then with the function `random.choice()` we pick the next character to add to the output (`output`) from the array associated with `currentgram` in the dictionary.

The default length of the generated text is arbitrarily set to 500 but this can be changed by supplying a value for `length`.

In [ ]:
import random
def markov_gen(seq, order, length=500, start=None):
    markov_dict = markov_process(seq, order)
    if start == None or len(start) != order or not (tuple(start) in markov_dict.keys()):
      currentgram = tuple(seq[:order])
    else:
      currentgram = tuple(start)
    output = list(currentgram)
    for i in range(length-order):
        next = random.choice(markov_dict[currentgram])
        if next == None:
          break;
        else:
          output.append(next)
        currentgram = tuple(output[-order:])
    return output

In [ ]:
output_text = ''.join(markov_gen(input_text, 3))
print(textwrap.fill(output_text, 120))

Alice seen suddenly a White nate!' (when the undering to that to hered at so very time tooked after a White Rabbit-hole
under after sistcoat-pocket, fore stupid), when her beginning by her the had neversations in time tired of the hot day
much out at to down mind to her befortunately was remarkable of it had peeped quite Rabbit of it whethe daisies, it all
seemed afterwards, when suddenly to do: on, Alice was nothing by her the that to take ought in the pleasures reading
nothis, but of sitting n


The `markov_gen()` allows a starting n-gram to be provided as a parameter. The provided n-gram must be of the same order as the n-grams constructed in the Markov chain and it must actually appear in the text. Otherwise the n-gram will be ignored and the function will return text generated using the first n-gram in the text.

In [ ]:
output_text = ''.join(markov_gen(input_text, 3, 1000, "Whi"))
print(textwrap.fill(output_text, 120))

White natural); but of sistcoat-pocket, and peeped to do: on the field at thered in tired at so versation?' So seemed on
thing a watch ought Alice wonder a daistcoat-pocket, and the there waisy-chain tired in would, fore of the Rabbit
picking by hed on thought Alice the thout pictuall seemed think its was rabbit over waisies, but with curred in the her
own mind look her. There seemed quite Rabbit-hole it of sisterwards, when her mind started on, Alice time it whethered
in the considering nothing to her the the had nothing nothis, what so versation?' So she time the thought the daistcoat-
pocket, or it so ver a watch out pink eyes ran close out of a was remarkable out of it of the day took her a day to get
very making up and (as she book a book,' thought to down actures remarkable ought it pop do: on, Alice way making nor
cons in time thought Alice ought Alice out of made her it, and started of that to getting to here seemed quite Rabbit
with picking then hurried on, Alice `withe book,'

# Kafkanstein

In this section we're going to explore how we can generate original text using a Markov chain by combining data sources. This exercise is inspired by the [Kafgenstein example](https://rednoise.org/rita/examples/p5/Kafgenstein/) developed for the RiTa library for Java/Processing and Javascript.

## Instructions

For this exercise, we will upload files to the Colab runtime.

> *Note that these files will be deleted when the runtime closes, either because the window is closed intentionally or due to inactivity. If this happens, you will need to reupload the files.*

Start by downloading the following files to your computer:

- [kafka.txt](https://rednoise.org/rita/examples/data/kafka.txt)
- [wittgenstein.txt](https://rednoise.org/rita/examples/data/wittgenstein.txt)

Next, upload these files to the Colab runtime: open the files area in the left sidebar on the left by clicking on the folder icon. Then upload the files by either (1) dragging and dropping the files from your computer into the Colab files area; or (2) clicking on the document icon to open a file dialog and select the files on your computer to upload.

We can now load the contents of these files into variables:

In [ ]:
kafka_file = open("/content/kafka.txt", 'r')
kafka_text = kafka_file.read()
# Print a short extract from the start of the file
print(textwrap.fill(kafka_text[:960], 120))

One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible
vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed
and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any
moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked.
"What's happened to me?" he thought. It wasn't a dream. His room, a proper human room although a little too small, lay
peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a
travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and
housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy
fur muff that cov


In [ ]:
wittgenstein_file = open("/content/wittgenstein.txt", 'r')
wittgenstein_text = wittgenstein_file.read()
# Print a short extract from the start of the file
print(textwrap.fill(wittgenstein_text[:960], 120))

Perhaps this book will be understood only by someone who has himself already had the thoughts that are expressed in it--
or at least similar thoughts.--So it is not a textbook.--Its purpose would be achieved if it gave pleasure to one person
who read and understood it.  The book deals with the problems of philosophy, and shows, I believe, that the reason why
these problems are posed is that the logic of our language is misunderstood. The whole sense of the book might be summed
up the following words: what can be said at all can be said clearly, and what we cannot talk about we must pass over in
silence.  Thus the aim of the book is to draw a limit to thought, or rather--not to thought, but to the expression of
thoughts: for in order to be able to draw a limit to thought, we should have to find both sides of the limit thinkable
i.e. we should have to be able to think what cannot be thought.  It will therefore only be in language that the limit
can


We can now generate a new text by supplying the `markov_gen()` function with a concatenation of the source texts. If we don't supply a starting n-gram (or one that is not of the required order) then the function will start with the first n-gram in the text, which will mean that it will start in the style of the first text used for the input, in the example here, the text from Kafka:

In [ ]:
kafkanstein_input = kafka_text + wittgenstein_text
kafkanstein_output = ''.join(markov_gen(kafkanstein_input, 10, 960))
print(textwrap.fill(kafkanstein_output, 120))

One morning, and it was out of the question whether I can get into the bedroom opened and Mr. Samsa appeared with these
apparently primitive ideas both the concept of a function cannot be said: it makes itself manifest. The world is a
metaphysical subject, or rather of showing that it is important but it is always a complex of objects could
corresponding to the evening she would not be the most sensible thing to do the job himself.  And without looking up
from what they signify two different resolution every time that his powerful chest shook.  So Gregor did not understands
propositions--the axioms of mechanics, for example, the propositions by combining them so as to form proposition. It
would require a justification, but I will work my way out of its primitive propositions: tautology and contradiction's
impossible, impossible to infer the existence and non-existence of an internal relation', 'internal relations.  Reality
is limited by the two w


If we supply a valid starting n-gram, we can have a bit more control over the starting text:

In [ ]:
kafkanstein_output = ''.join(markov_gen(kafkanstein_input, 10, 960, "philosophy"))
print(textwrap.fill(kafkanstein_output, 120))

philosophy of psychology. Does not my study of thoughts: for in order to signify something to show the logic of our
language. Proposition, then we have determinate relations obtain: rather, that characterizes its sense with reality, in
order that something similar. No-one drank very much. Gregor only remained from his business considerate as he could,
but, unfortunately, to put Gregor's sister also had to help too - he could watch the family was totally by surprise, no-
one was ever ill but that was not clear whether its meaning only in so far as a proposition. The negating propositions
of logic be irrefutable, but obvious, just as, for instance by writing 'fxg'--that would exchange a tired grin. With a
kind of attack. Instead of, 'This proposition contradict one another like the soughing of the propositions of logic.The
truth of a propositions that such internal relation'. I introduce these expressions have the answers to question to
leave him no


# Generating text using word n-grams

The `markov_process()` and `markov_gen()` functions can handle general sequences, not just strings. So if we process the text files to break them down into sequences of words, rather than characters, we can build word-based n-gram models and generate text using whole words. In general, this means that we need longer pieces of text to have enough variety, but the Kafka and Wittgenstein texts will suffice.

To generate the word sequences we just need to split the string. The simplest way to do this is by split on space `' '` characters:

In [ ]:
kafka_words = kafka_text.split(' ')

We can then generate new text from this sequence, we simply need to remember to join the result using a space character to get back to something like the source text:

In [ ]:
kafka_output = ' '.join(markov_gen(kafka_words, 3, 100))
print(textwrap.fill(kafka_output, 120))

One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed when Gregor came back
from his business trips, who would receive him sitting in the armchair in his nightgown when he came in from the hall,
could see straight away that Gregor had been bringing home every month, keeping only a little for himself, so that that,
too, had been accumulating. Behind the door, Gregor nodded with enthusiasm in his pleasure at this unexpected thrift and
caution. He could actually have used this surplus money to reduce his father's debt to his boss, and


We can also still supply a starting, but we must remember to split any string into a sequence:

In [ ]:
kafka_output = ' '.join(markov_gen(kafka_words, 3, 100, "Samsa woke from".split()))
print(textwrap.fill(kafka_output, 120))

Samsa woke from troubled dreams, he found himself transformed in his bed when Gregor came back from his business trips,
who would receive him sitting in the armchair in his nightgown when he came in from the street towards the stairway, the
curtains flew up, the newspapers on the table at home for the benefit of his astonished and delighted family. They had
been good times and they had never asked each other about their work but all three had jobs which were very good and
held particularly good promise for the future. Gregor, though, did think about the future.


# Experimenting with the model

Here are some ideas for experimenting with the Markov chain model developed here:

- Experiment with the value for `order` to see how this changes the quality of the text produced for both character-based generation and word-based generation
- Experiment with different input texts by uploading them to your runtime and then using them to generate new texts
- Experiment with the length of the input text and how this impacts the diversity of output texts
- Experiment with combining different texts together to form interesting mixtures
- Experiment with more structured forms of text (e.g. poetry), or data files and comment on the success of the markov model to generate valid forms of these files

## Tutorial Assignment

Attempt at least 2 of the above suggestions to experiment with the model, or try some experiments of your own.

Report on your experiments by submitting a notebook with the code for your experiments and comments on the things that you tried, what worked better than expected, what worked worse.

Include in your submission any text files that your notebook relies on.